In [0]:
import pandas as pd
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
pip install --upgrade google-cloud-bigquery

     |████████████████████████████████| 174kB 4.9MB/s 
     |████████████████████████████████| 92kB 5.4MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.7.2, but you'll have google-auth 1.14.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.14.0 which is incompatible.
ERROR: google-cloud-storage 1.18.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 0.5.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: google-auth 1.7.2
    Uninstalling google-auth-1.7.2:
      Successfully uninstalled google-auth-1.7.2
  Found existing installation: 

In [0]:
# pip install --upgrade google-cloud-bigquery

from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    '/content/drive/My Drive/ML_works/Internet_sathi/IS_ML_PROBLEMS/infra-211714-fa79bf90e271.json')
project_id = 'infra-211714'
client = bigquery.Client(credentials= credentials,project=project_id)

def normalise(a,b,c):
  x=min(a)
  y=max(a)
  if y-x!=0:
    norm_list=[round((((i-x)/(y-x))*(c-b)+b),2) for i in a]
    normalised=[100-i for i in norm_list]
  else:
    normalised=[0]
  return normalised

def bias_parameter_for_saathi(table_name, answer_table_name, form_id, device_id):

  original_data = client.query('''
        WITH   cfa AS(   SELECT     id,     ARRAY_AGG( fa     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)  
        FROM     `infra-211714.is_dashboard.{secondary}` AS fa   GROUP BY     id,     fa.question,     fa.answer,   
          fa.nested_question,     fa.nested_answer,     fa.parent_value),   cf AS(   SELECT     id,     ARRAY_AGG( fa     ORDER BY      
            modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)   FROM     `infra-211714.is_dashboard.{primary}` AS fa 
              GROUP BY     id),   fl AS (   SELECT     id,     ARRAY_AGG( fl     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     
              (0)].* EXCEPT (id)   FROM     `infra-211714.is_dashboard.form_label` AS fl   GROUP BY     id,     fl.ORDER,     fl.value,     fl.formId),
                fq AS (   SELECT     id,     ARRAY_AGG( fq     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id) 
                  FROM     `infra-211714.is_dashboard.formquestions` AS fq   GROUP BY     id,     fq.formId,     fq.question,     fq.LANGUAGE,   
                    fq.title),   sf AS(   SELECT     id,     ARRAY_AGG( sf     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET   
                      (0)].* EXCEPT (id)   FROM     `infra-211714.is_dashboard.saathi_form` AS sf   GROUP BY     id),   sfv AS(   SELECT     id,   
                        ARRAY_AGG( sfv     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)   FROM   
                          `infra-211714.is_dashboard.saathiformvillages` AS sfv   GROUP BY     id,     sfv.stateId),   st AS(   SELECT     id,   
                            ARRAY_AGG( state     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)   FROM   
                              `infra-211714.is_dashboard.states` AS state   GROUP BY     id),   dt AS(   SELECT     id,     ARRAY_AGG( district   
                                ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)   FROM     
                                `infra-211714.is_dashboard.districts` AS district   GROUP BY     id),   p AS(   SELECT     id,   
                                  ARRAY_AGG( pt     ORDER BY       modifiedAt DESC     LIMIT       1 )[   OFFSET     (0)].* EXCEPT (id)   FROM  
                                      `infra-211714.is_dashboard.partner` AS pt   GROUP BY     id)


        SELECT
          fl.label AS AnswerLabel,
          fq.title AS Title,
          fl.ORDER AS QuestionOrder,
          fl.formId,
          cf.deviceId
        FROM
          cf
        JOIN
          cfa
        ON
          cf.id=cfa.id
        LEFT OUTER JOIN
          sf
        ON
          sf.deviceId = cf.deviceId
        JOIN
          sfv
        ON
          SFV.id =sf.id
        JOIN
          fq
        ON
          fq.question = cfa.question
        JOIN
          fl
        ON
          fl.ORDER = cfa.question
        LEFT OUTER JOIN
          st
        ON
          st.id = sfv.stateId
        LEFT OUTER JOIN
          dt
        ON
          dt.id = sfv.districtId
        LEFT OUTER JOIN
          p
        ON
          p.id = cf.partner
        WHERE
          fq.LANGUAGE='en'
          and cf.formId={input_fid}
          AND fq.formId= cf.formId
          AND fl.value = cfa.answer
          AND fl.formId = cf.formId
          AND sf.modifiedAT= sfv.modifiedAt
          AND cfa.modifiedAt = cf.modifiedAt
          AND sf.formId = cf.formId
          AND cf.deviceId="{input_did}"
        '''.format(primary=str(table_name),secondary=str(answer_table_name), input_fid=form_id, input_did=str(device_id))).result().to_dataframe()

  qns=list(original_data.Title.unique())
  bp_saathi=[]
  for m in qns:
    qns_data=original_data[original_data.Title==m]
    total_options=len(qns_data.AnswerLabel.value_counts())
    unbiased_percentage=100/total_options
    options=list(qns_data.AnswerLabel.value_counts().index)
    bias_param=[]
    for i in options:
      if ((qns_data.AnswerLabel.value_counts()[str(i)]/len(qns_data))*100-unbiased_percentage>0)&(total_options>0):
        bias_param.append(((qns_data.AnswerLabel.value_counts()[str(i)]/len(qns_data))*100-unbiased_percentage)*(total_options-1))
      else:
        bias_param.append(0)
    bp_saathi.append(int(np.mean(bias_param)))

  if len(bp_saathi)>0:
    quality=int(np.mean(normalise(bp_saathi,0,100)))
  else:
    quality=0
  return quality

In [6]:
bias_parameter_for_saathi("clientformdatas_tataamc","clientformdatas_answer_tataamc",71,"5d19e5600bd83f311058ccef")

0

In [0]:
def normalize(a,b,c):
  x=min(a)
  y=max(a)
  norm_list=[round((((i-x)/(y-x))*(c-b)+b),2) for i in a]
  # normalised=[100-i for i in norm_list]
  return norm_list

In [0]:
import numpy as np

In [0]:
k=np.zeros((4,2,3))

In [0]:
k.shape[2]

3

In [0]:
for m in range(k.shape[2]):
  print(k[:,:,m])

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [7]:
3*8

24

In [8]:
np.zeros((3,4))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [0]:
a=dict(zip(["a","b"],[3,4]))

In [23]:
list(a.keys())[0]

'a'

In [0]:
a=np.array([3,4,5])

In [0]:
b=np.array([4,5,6])

In [0]:
X = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

In [29]:
X[0]

'I am going to the bar tonight'